In [1]:
from valurap2 import path_planning, gcode

input_fn = "diplo.gcode"
seg_no = 18

#input_fn = "box20.gcode"
#seg_no = 18

lines = gcode.reader(input_fn)
pg = gcode.path_gen(lines)
sg = gcode.gen_segments(pg)
for i, s in enumerate(sg):
    if isinstance(s, gcode.do_segment):
        print("segment", i, len(s.path))
        
    if i == seg_no:
        break
        
print(s.path)

Extruder switch to 0
do_extruder
Do Home
do_home
Set current ['E0']
Set current ['E0']
Set current ['E0']
Set current ['E0']
segment 6 663
segment 8 6
segment 10 1659
segment 12 5
segment 14 1626
segment 16 7
segment 18 2270
[[-7.895, 16.607000000000003, 0, -237.33058, 4000], [-8.207, 15.999000000000004, 75.0, -237.36468, 4001], [-8.41, 15.525000000000004, 75.0, -237.3904, 4002], [-8.651, 14.981000000000003, 75.0, -237.42008, 4003], [-8.845, 14.466000000000003, 75.0, -237.44754, 4004], [-9.113, 13.924000000000003, 75.0, -237.47771, 4005], [-9.383, 13.431000000000003, 75.0, -237.50575, 4006], [-9.565, 13.016000000000004, 75.0, -237.52836, 4007], [-9.822, 12.419000000000004, 75.0, -237.56078, 4008], [-10.009, 11.952000000000004, 75.0, -237.58588, 4009], [-10.164, 11.530000000000003, 75.0, -237.60831, 4010], [-10.343, 10.961000000000004, 75.0, -237.63807, 4011], [-10.58, 10.343000000000004, 75.0, -237.67109, 4012], [-10.835, 9.794000000000004, 75.0, -237.70129, 4013], [-10.99, 9.390000000

In [2]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
from matplotlib import collections  as mc

%matplotlib notebook


path = np.array(s.path)

df = pd.DataFrame({"x": path[:, 0], "y": path[:, 1], "v": path[:,2]})
df["dx"] = df["x"] - df["x"].shift(1)
df["dy"] = df["y"] - df["y"].shift(1)
df["next_dx"] = df["dx"].shift(-1)
df["next_dy"] = df["dy"].shift(-1)
df["d"] = np.linalg.norm([df["dx"], df["dy"]], axis=0)
df["dt"] = df["d"] / df["v"]

if 0:
    def add_angle(df):
        df["angle"] = 180 - np.arccos((df["dx"]*df["next_dx"] + df["dy"]*df["next_dy"])/np.linalg.norm([df["dx"], df["dy"]], axis=0)/np.linalg.norm([df["next_dx"], df["next_dy"]], axis=0)) * 180 / math.pi

    def add_circle(df):
        x1 = df["x"].shift(1)
        y1 = df["y"].shift(1)
        x2 = df["x"]
        y2 = df["y"]
        x3 = df["x"].shift(-1)
        y3 = df["y"].shift(-1)

        x12 = x1 - x2;  
        x13 = x1 - x3;  

        y12 = y1 - y2;  
        y13 = y1 - y3;  

        y31 = y3 - y1;  
        y21 = y2 - y1;  

        x31 = x3 - x1;  
        x21 = x2 - x1;  

        sx13 = x1 * x1 - x3 * x3 
        sy13 = y1 * y1 - y3 * y3  

        sx21 = x2 * x2 - x1 * x1;  
        sy21 = y2 * y2 - y1 * y1;  

        f = (sx13 * x12 + sy13 * x12 + sx21 * x13 + sy21 * x13) / (2 * (y31 * x12 - y21 * x13))              
        g = (sx13 * y12 + sy13 * y12 + sx21 * y13 + sy21 * y13) / (2 * (x31 * y12 - x21 * y13))
        c = -x1 * x1 - y1 * y1 - 2 * g * x1 - 2 * f * y1

        cx = -g 
        cy = -f

        r = np.sqrt(cx * cx + cy * cy - c)

        v1x = x1 - cx
        v2x = x2 - cx
        v3x = x3 - cx

        v1y = y1 - cy
        v2y = y2 - cy
        v3y = y3 - cy

        v12x = (v1x + v2x) / 2
        v12y = (v1y + v2y) / 2
        d12 = r - np.linalg.norm([v12x, v12y], axis=0)
        p12x = cx + v12x / (r - d12) * r
        p12y = cy + v12y / (r - d12) * r

        v23x = (v2x + v3x) / 2
        v23y = (v2y + v3y) / 2
        d23 = r - np.linalg.norm([v23x, v23y], axis=0)
        p23x = cx + v23x / (r - d23) * r
        p23y = cy + v23y / (r - d23) * r

        df["r"] = r
        df["cx"] = cx
        df["cy"] = cy
        df["d12"] = d12
        df["d23"] = d23
        df["p12x"] = p12x
        df["p12y"] = p12y
        df["p23x"] = p23x
        df["p23y"] = p23y

    add_angle(df)
    add_circle(df)

    sdf = df[20:-20]
    lines = []
    colors = []
    for l in sdf.iterrows():
        l = l[1]
        lines.append(((l.x, l.y),(l.cx, l.cy)))
        colors.append([0,1,0])
        lines.append(((l.cx, l.cy),(l.p12x, l.p12y)))
        colors.append([1,0,0])
        lines.append(((l.cx, l.cy),(l.p23x, l.p23y)))
        colors.append([1,1,0])

    lc = mc.LineCollection(lines, colors=colors, linewidths=1)

    display(sdf)
    ax = sdf.plot(x="x", y="y", style=".-")
    ax.add_collection(lc)


    plt.axis("square")


    #display(p.ax)

In [3]:
pp = path_planning.PathPlanner()
pp.accel_step = 1000
pp.max_xa = 4000
pp.max_ya = 4000
pp.set_mode("print")


path = s.path
path, slowdowns = pp.make_path(path, speed_k=4)
slowdowns, updated, cc = pp.process_corner_errors(path, slowdowns)
slowdowns, updated = pp.reverse_pass(path, slowdowns)
slowdowns, updated = pp.forward_pass(path, slowdowns)
_, updated = pp.forward_pass(path, slowdowns)

assert updated == 0

speeds = pp.gen_speeds(path, slowdowns)
_, _, cc = pp.process_corner_errors(path, slowdowns)

plato = pd.DataFrame()
plato["middle_x"] = path["px"] + cc["mdx"]
plato["middle_y"] = path["py"] + cc["mdy"]
plato["middle_e"] = path["pe"]
plato["middle_vx"] = cc["mvx"]
plato["middle_vy"] = cc["mvy"]
plato["middle_v"] = np.linalg.norm(cc[["mvx", "mvy"]], axis=1)

plato["start_x"] = path["px"] + speeds["unit_x"] * cc["l_entry"]
plato["start_y"] = path["py"] + speeds["unit_y"] * cc["l_entry"]
plato["start_e"] = path["pe"] + speeds["unit_e"] * cc["l_entry"]
plato["start_vx"] = speeds["entry_x"]
plato["start_vy"] = speeds["entry_y"]
plato["start_v"] = speeds["entry"]
plato["top_vx"] = speeds["plato_x"]
plato["top_vy"] = speeds["plato_y"]
plato["top_v"] = speeds["plato"]
plato["end_x"] = path["x"] - speeds["unit_x"] * cc["l_exit"]
plato["end_y"] = path["y"] - speeds["unit_y"] * cc["l_exit"]
plato["end_e"] = path["e"] - speeds["unit_e"] * cc["l_exit"]
plato["end_vx"] = speeds["exit_x"]
plato["end_vy"] = speeds["exit_y"]
plato["end_v"] = speeds["exit"]
plato["v_to_ve"] = speeds["v_to_ve"]

plato["pre_x"] = plato["end_x"].shift(1).fillna(path["px"][0])
plato["pre_y"] = plato["end_y"].shift(1).fillna(path["py"][0])
plato["pre_e"] = plato["end_e"].shift(1).fillna(0)
plato["pre_vx"] = plato["end_vx"].shift(1).fillna(0)
plato["pre_vy"] = plato["end_vy"].shift(1).fillna(0)
plato["pre_v"] = plato["end_v"].shift(1).fillna(0)
plato["pre_v_to_ve"] = plato["v_to_ve"].shift(1).fillna(0)
plato["corner_dt"] = cc["dt"]

plato["plato_l"] = cc["l_free"]
print("pow:", pow)
plato["max_top_v"] = np.sqrt(2 * speeds["max_a"] * 0.9 * plato["plato_l"] + pow(plato["start_v"], 2) + pow(plato["end_v"], 2)) / np.sqrt(2)

assert all(plato["max_top_v"] >= plato["start_v"])
assert all(plato["max_top_v"] >= plato["end_v"])

plato["real_top_v"] = np.min(plato[["top_v", "max_top_v"]], axis=1)
plato["real_top_vx"] = plato["real_top_v"] * speeds["unit_x"]
plato["real_top_vy"] = plato["real_top_v"] * speeds["unit_y"]
plato["accel_dt"] = (plato["real_top_v"] - plato["start_v"]) / speeds["max_a"]
plato["decel_dt"] = (plato["real_top_v"] - plato["end_v"]) / speeds["max_a"]
plato["accel_l"] = (plato["real_top_v"] + plato["start_v"]) * plato["accel_dt"] / 2
plato["decel_l"] = (plato["real_top_v"] + plato["end_v"]) * plato["decel_dt"] / 2
plato["top_l"] = plato["plato_l"] - plato["accel_l"] - plato["decel_l"]
plato["top_dt"] = (plato["top_l"] / plato["real_top_v"]).fillna(0)
plato["dt"] = plato["corner_dt"] + plato["accel_dt"] + plato["top_dt"] + plato["decel_dt"]
plato["t"] = np.cumsum(plato["dt"]).shift(1).fillna(0)
#display(plato)
#display(plato.iloc[0])
#display(plato.iloc[1])
#display(plato.iloc[-2])
#display(plato.iloc[-1])

pre_corner_segs = pd.DataFrame()
pre_corner_segs["idx"] = plato.index.values * 5
pre_corner_segs["tp"] = 0
pre_corner_segs["line"] = path["line"]
pre_corner_segs["t"] = plato["t"]

dt = plato["corner_dt"] / 2
dt[0] = 0
dt[len(dt) - 1] *= 2 
pre_corner_segs["dt"] = dt

pre_corner_segs["start_x"] = plato["pre_x"]
pre_corner_segs["start_y"] = plato["pre_y"]
pre_corner_segs["start_e"] = plato["pre_e"]
pre_corner_segs["start_vx"] = plato["pre_vx"]
pre_corner_segs["start_vy"] = plato["pre_vy"]
pre_corner_segs["end_x"] = plato["middle_x"]
pre_corner_segs["end_y"] = plato["middle_y"]
pre_corner_segs["end_e"] = plato["middle_e"]
pre_corner_segs["end_vx"] = plato["middle_vx"]
pre_corner_segs["end_vy"] = plato["middle_vy"]
pre_corner_segs["v_to_ve"] = plato["pre_v_to_ve"]

corner_segs = pd.DataFrame()
corner_segs["idx"] = pre_corner_segs["idx"] + 1
corner_segs["tp"] = 1
corner_segs["line"] = path["line"]
corner_segs["t"] = pre_corner_segs["t"] + pre_corner_segs["dt"]

dt = plato["corner_dt"] / 2
dt[0] *= 2
dt[len(dt) - 1] = 0 
corner_segs["dt"] = dt

corner_segs["start_x"] = pre_corner_segs["end_x"]
corner_segs["start_y"] = pre_corner_segs["end_y"]
corner_segs["start_e"] = pre_corner_segs["end_e"]
corner_segs["start_vx"] = pre_corner_segs["end_vx"]
corner_segs["start_vy"] = pre_corner_segs["end_vy"]
corner_segs["end_x"] = plato["start_x"]
corner_segs["end_y"] = plato["start_y"]
corner_segs["end_e"] = plato["start_e"]
corner_segs["end_vx"] = plato["start_vx"]
corner_segs["end_vy"] = plato["start_vy"]
corner_segs["v_to_ve"] = plato["v_to_ve"]

accel_segs = pd.DataFrame()
accel_segs["idx"] = corner_segs["idx"] + 1
accel_segs["tp"] = 2
accel_segs["line"] = path["line"]
accel_segs["t"] = corner_segs["t"] + corner_segs["dt"]
accel_segs["dt"] = plato["accel_dt"]
accel_segs["start_x"] = corner_segs["end_x"]
accel_segs["start_y"] = corner_segs["end_y"]
accel_segs["start_e"] = corner_segs["end_e"]
accel_segs["start_vx"] = corner_segs["end_vx"]
accel_segs["start_vy"] = corner_segs["end_vy"]
accel_segs["end_x"] = accel_segs["start_x"] + speeds["unit_x"] * plato["accel_l"]
accel_segs["end_y"] = accel_segs["start_y"] + speeds["unit_y"] * plato["accel_l"]
accel_segs["end_e"] = accel_segs["start_e"] + speeds["unit_e"] * plato["accel_l"]
accel_segs["end_vx"] = plato["real_top_vx"]
accel_segs["end_vy"] = plato["real_top_vy"]
accel_segs["v_to_ve"] = plato["v_to_ve"]

top_segs = pd.DataFrame()
top_segs["idx"] = accel_segs["idx"] + 1
top_segs["tp"] = 3
top_segs["line"] = path["line"]
top_segs["t"] = accel_segs["t"] + accel_segs["dt"]
top_segs["dt"] = plato["top_dt"]
top_segs["start_x"] = accel_segs["end_x"]
top_segs["start_y"] = accel_segs["end_y"]
top_segs["start_e"] = accel_segs["end_e"]
top_segs["start_vx"] = accel_segs["end_vx"]
top_segs["start_vy"] = accel_segs["end_vy"]
top_segs["end_x"] = top_segs["start_x"] + speeds["unit_x"] * plato["top_l"]
top_segs["end_y"] = top_segs["start_y"] + speeds["unit_y"] * plato["top_l"]
top_segs["end_e"] = top_segs["start_e"] + speeds["unit_e"] * plato["top_l"]
top_segs["end_vx"] = top_segs["start_vx"]
top_segs["end_vy"] = top_segs["start_vy"]
top_segs["v_to_ve"] = plato["v_to_ve"]

decel_segs = pd.DataFrame()
decel_segs["idx"] = top_segs["idx"] + 1
decel_segs["tp"] = 4
decel_segs["line"] = path["line"]
decel_segs["t"] = top_segs["t"] + top_segs["dt"]
decel_segs["dt"] = plato["decel_dt"]
decel_segs["start_x"] = top_segs["end_x"]
decel_segs["start_y"] = top_segs["end_y"]
decel_segs["start_e"] = top_segs["end_e"]
decel_segs["start_vx"] = top_segs["end_vx"]
decel_segs["start_vy"] = top_segs["end_vy"]
decel_segs["end_x"] = plato["end_x"]
decel_segs["end_y"] = plato["end_y"]
decel_segs["end_e"] = plato["end_e"]
decel_segs["end_vx"] = plato["end_vx"]
decel_segs["end_vy"] = plato["end_vy"]
decel_segs["v_to_ve"] = plato["v_to_ve"]

pre_corner_segs = pre_corner_segs.set_index("idx")
corner_segs = corner_segs.set_index("idx")
accel_segs = accel_segs.set_index("idx")
top_segs = top_segs.set_index("idx")
decel_segs = decel_segs.set_index("idx")

all_segs_raw = pd.concat([pre_corner_segs, corner_segs, accel_segs, top_segs, decel_segs]).sort_index()
all_segs_filter = (all_segs_raw["dt"] > 1e-5).values
all_segs_filter[0] = True
all_segs_filter[-1] = True
all_segs = all_segs_raw[all_segs_filter][[
    "tp", "line", "t", "v_to_ve", "start_x", "start_y", "start_e", "start_vx", "start_vy",
]].copy().sort_index()

all_segs["start_v"] = np.linalg.norm(all_segs[["start_vx", "start_vy"]], axis = 1)
all_segs["end_x"] = all_segs["start_x"].shift(-1)
all_segs["end_y"] = all_segs["start_y"].shift(-1)
all_segs["end_e"] = all_segs["start_e"].shift(-1)
all_segs["end_vx"] = all_segs["start_vx"].shift(-1)
all_segs["end_vy"] = all_segs["start_vy"].shift(-1)
all_segs["end_v"] = all_segs["start_v"].shift(-1)
all_segs["dt"] = all_segs["t"].shift(-1) - all_segs["t"]

all_segs = all_segs.iloc[1:-1,:]

all_segs["avg_v"] = (all_segs["start_v"] + all_segs["end_v"]) / 2

de = all_segs["end_e"] - all_segs["start_e"]
de[np.abs(de) < 1e-5] = 0
avg_ve = de / all_segs["dt"]

all_segs["dx"] = np.abs(80 * (all_segs["end_x"] - all_segs["start_x"]))
all_segs["dy"] = np.abs(80 * (all_segs["end_y"] - all_segs["start_y"]))
all_segs["de"] = np.abs(837 * de)
all_segs["dx_dt"] = all_segs["dt"] / all_segs["dx"] * 1000000
all_segs["dy_dt"] = all_segs["dt"] / all_segs["dy"] * 1000000
all_segs["de_dt"] = all_segs["dt"] / all_segs["de"] * 1000000
all_segs["avg_ve"] = avg_ve
all_segs["v_to_ve2"] = (all_segs["avg_ve"] / all_segs["avg_v"]).fillna(0)

all_segs                                  
            

cc updated 2049
rp updated: 1433
fp updated: 654
pow: <built-in function pow>


,tp,line,t,v_to_ve,start_x,start_y,start_e,start_vx,start_vy,start_v,...,dt,avg_v,dx,dy,de,dx_dt,dy_dt,de_dt,avg_ve,v_to_ve2
idx,,,,,,,,,,,,,,,,,,,,,
1,1,4001,0.000000,-0.049899,-7.895000,16.607000,0.000000,0.000000,0.000000,0.000000,...,0.009589,21.555293,7.549214,14.711288,8.632508,1270.175245,651.800455,1110.780862,-1.075589,-0.049899
2,2,4001,0.009589,-0.049899,-7.989365,16.423109,-0.010314,-19.682323,-38.355297,43.110586,...,0.005197,54.793830,10.401331,20.269261,11.893898,499.673044,256.411167,436.969037,-2.734160,-0.049899
3,3,4001,0.014786,-0.049899,-8.119382,16.169743,-0.024524,-30.350394,-59.144357,66.477074,...,0.000678,66.477074,1.645375,3.206373,1.881483,411.856273,211.347298,360.172400,-3.317142,-0.049899
4,4,4001,0.015464,-0.049899,-8.139949,16.129663,-0.026772,-30.350394,-59.144357,66.477074,...,0.001943,62.109980,4.407048,8.588093,5.039448,440.814822,226.207606,385.496938,-3.099229,-0.049899
5,0,4002,0.017406,-0.049899,-8.195037,16.022312,-0.032793,-26.362763,-51.373589,57.742886,...,0.000454,57.725466,0.924085,1.880471,1.094364,491.058752,241.311990,414.652066,-2.881315,-0.049914
6,1,4002,0.017860,-0.049880,-8.206588,15.998806,-0.034100,-24.547642,-52.226733,57.708045,...,0.000454,57.725466,0.858192,1.911442,1.093940,528.762976,237.401999,414.812664,-2.880199,-0.049895
7,2,4002,0.018314,-0.049880,-8.217316,15.974913,-0.035407,-22.732521,-53.079878,57.742886,...,0.004596,67.741815,9.805073,22.894604,12.997542,468.709833,200.734380,353.584865,-3.378943,-0.049880
8,3,4002,0.022910,-0.049880,-8.339879,15.688731,-0.050936,-30.605382,-71.462814,77.740744,...,0.000620,77.740744,1.519224,3.547351,2.013874,408.424892,174.916146,308.107170,-3.877687,-0.049880
9,4,4002,0.023530,-0.049880,-8.358869,15.644389,-0.053342,-30.605382,-71.462814,77.740744,...,0.001657,74.136558,3.867944,9.031554,5.127322,428.280674,183.419782,323.085956,-3.697911,-0.049880


In [4]:
if 0:
    all_segs.plot(x="t", y=["avg_v", "v_to_ve", "v_to_ve2"], secondary_y=["v_to_ve", "v_to_ve2"])

In [5]:
if 0:
    all_segs.plot(x="start_x", y="start_y", style=".-")
    plt.axis("square")



In [6]:
from math import ceil

track_t = []
track_x = []
track_y = []
track_e = []
track_e_exp = []
track_vx = []
track_vy = []
track_ve = []

last_e = 0
last_ve = 0
borders = 100

delta_t = 0.001

# first and last segments require patching, as j-only move takes 1.5x time
dt_first = all_segs["dt"].iloc[0]
extra_start = int(ceil(dt_first / delta_t)) + borders
last_t = -extra_start * delta_t

t_last = all_segs["t"].iloc[-1]
dt_last = all_segs["dt"].iloc[-1]
extra_end = int(ceil(dt_last / delta_t)) + borders
final_t = t_last + extra_end * delta_t

last_i = len(all_segs) - 1

for i, seg in enumerate(all_segs.itertuples()):
    t = seg.t
    dt = seg.dt
    end_t = t + dt

    st_x = seg.start_x
    st_vx = seg.start_vx
    tg_x = seg.end_x
    tg_vx = seg.end_vx

    dvx = tg_vx - st_vx
    ax = dvx / dt
    jx = 0

    st_y = seg.start_y
    st_vy = seg.start_vy
    tg_y = seg.end_y
    tg_vy = seg.end_vy

    dvy = tg_vy - st_vy
    ay = dvy / dt
    jy = 0
    
    st_e = seg.start_e
    tg_e = seg.end_e

    st_v = math.sqrt(st_vx * st_vx + st_vy * st_vy)
    tg_v = math.sqrt(tg_vx * tg_vx + tg_vy * tg_vy)
    st_ve = st_v * seg.v_to_ve
    tg_ve = tg_v * seg.v_to_ve

    if seg.tp in (0, 1):
        st_ve = tg_ve = (tg_e - st_e) / dt

    dve = tg_ve - st_ve
    ae = dve / dt
    je = 0

    calc_j = False
    if i == 0:
        dt *= 1.5
        t = end_t - dt
        
        tg_ve += st_ve
        st_ve = 0
        calc_j = True
    elif i == last_i:
        dt *= 1.5
        end_t = t + dt
        
        st_ve += tg_ve
        tg_ve = 0
        calc_j = True

    if calc_j:
        dx = tg_x - st_x
        dy = tg_y - st_y
        de = tg_e - st_e
        
        jx = -(12 * dx - 6 * dt * tg_vx - 6 * dt * st_vx) / dt / dt / dt
        ax = (6 * dx - 2 * dt * tg_vx - 4 * dt * st_vx) / dt / dt

        jy = -(12 * dy - 6 * dt * tg_vy - 6 * dt * st_vy) / dt / dt / dt
        ay = (6 * dy - 2 * dt * tg_vy - 4 * dt * st_vy) / dt / dt

        je = -(12 * de - 6 * dt * tg_ve - 6 * dt * st_ve) / dt / dt / dt
        ae = (6 * de - 2 * dt * tg_ve - 4 * dt * st_ve) / dt / dt
        
        print(ax, jx, ay, jy, ae, je)
                
    dx = st_vx * dt + ax * dt * dt / 2 + jx * dt * dt * dt / 6
    dx_err =  dx - (tg_x - st_x)

    dy = st_vy * dt + ay * dt * dt / 2 + jy * dt * dt * dt / 6
    dy_err =  dy - (tg_y - st_y)

    de = st_ve * dt + ae * dt * dt / 2 + je * dt * dt * dt / 6
    de_err =  de - (tg_e - st_e)

    try:
        assert abs(dx_err) < 1e-10
        assert abs(dy_err) < 1e-10
        assert abs(de_err) < 1e-10
    except:
        print(t, seg.tp, seg.line, dx_err, dy_err, de_err)
        raise
        
    if i == 0:
        while last_t < t:
            track_t.append(last_t)
            track_x.append(st_x)
            track_y.append(st_y)
            track_vx.append(st_vx)
            track_vy.append(st_vy)
            track_e.append(st_e)
            track_ve.append(st_ve)

            last_t += delta_t

    while last_t < end_t:
        ct = last_t - t
        dx = st_vx * ct + ax * ct * ct / 2 + jx * ct * ct * ct / 6
        dy = st_vy * ct + ay * ct * ct / 2 + jy * ct * ct * ct / 6
        de = st_ve * ct + ae * ct * ct / 2 + je * ct * ct * ct / 6
        x = st_x + dx
        y = st_y + dy
        e = st_e + de
        vx = st_vx + ax * ct + jx * ct * ct / 2
        vy = st_vy + ay * ct + jy * ct * ct / 2
        ve = st_ve + ae * ct + je * ct * ct / 2
        
        track_t.append(last_t)
        track_x.append(x)
        track_y.append(y)
        track_vx.append(vx)
        track_vy.append(vy)
        track_e.append(e)
        track_ve.append(ve)
        
        last_t += delta_t
        
while last_t < final_t:
    track_t.append(last_t)
    track_x.append(tg_x)
    track_y.append(tg_y)
    track_e.append(tg_e)
    track_vx.append(tg_vx)
    track_vy.append(tg_vy)
    track_ve.append(tg_ve)
    last_t += delta_t

df = pd.DataFrame()
df["t"] = track_t
df["x"] = track_x
df["y"] = track_y
df["e"] = track_e
df["vx"] = track_vx
df["vy"] = track_vy
df["ve"] = track_ve
df["v"] = np.linalg.norm(df[["vx", "vy"]], axis=1)
df["dx"] = (df["x"] - df["x"].shift(1)) * 80
df["dy"] = (df["y"] - df["y"].shift(1)) * 80
df["d"] = np.linalg.norm(df[["dx", "dy"]], axis=1)
df["de"] = - (df["e"] - df["e"].shift(1)) * 837

if 1:
    display(df)


-5.366572433003852e-13 -190279.9924417809 -1.6099717299011558e-11 -370802.03655320976 3.3541077706274076e-14 -20796.6273790506
-5333.333333333336 67044.19851250232 -5333.333333333334 67044.19851250232 374.1625502739669 -4703.517805596079


,t,x,y,e,vx,vy,ve,v,dx,dy,d,de
0,-0.110,-7.895,16.607,0.00000,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,-0.109,-7.895,16.607,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
2,-0.108,-7.895,16.607,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
3,-0.107,-7.895,16.607,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
4,-0.106,-7.895,16.607,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
5,-0.105,-7.895,16.607,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
6,-0.104,-7.895,16.607,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
7,-0.103,-7.895,16.607,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
8,-0.102,-7.895,16.607,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
9,-0.101,-7.895,16.607,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0


In [7]:
if 0:
    df.plot(x="x", y="y", style=".-")
    plt.axis("square")

In [8]:
df["de_smooth"] = - df["de"].rolling(20, center=True, win_type="hamming").mean() / 837
df["e_smooth"] = df["de_smooth"].cumsum()

df["de_adv"] = df["de_smooth"] * 5

df["e_adv"] = df["e_smooth"] + df["de_adv"]
df["e_sm_d"] = - (df["e_smooth"] - df["e_smooth"].shift(1)) * 837
df["e_adv_d"] = - (df["e_adv"] - df["e_adv"].shift(1)) * 837

if 0:
    df.plot(x="t", y=["de", "e_sm_d", "e_adv_d", "e", "e_adv"], secondary_y=["e", "e_adv"], style=".-")

In [9]:
df["dx_smooth"] = df["dx"].rolling(5, center=True, win_type="hamming").mean() / 80
df["dy_smooth"] = df["dy"].rolling(5, center=True, win_type="hamming").mean() / 80
df["x_smooth"] = df["dx_smooth"].cumsum() + df["x"].iloc[0]
df["y_smooth"] = df["dy_smooth"].cumsum() + df["y"].iloc[0]

if 0:
    df["x_smooth_d"] = (df["x_smooth"] - df["x_smooth"].shift(1)) * 80
    df["y_smooth_d"] = (df["y_smooth"] - df["y_smooth"].shift(1)) * 80
    df["x_smooth_dd"] = df["x_smooth_d"] - df["x_smooth_d"].shift(1)
    df["y_smooth_dd"] = df["y_smooth_d"] - df["y_smooth_d"].shift(1)
    df["x_smooth_ddd"] = df["x_smooth_dd"] - df["x_smooth_dd"].shift(1)
    df["y_smooth_ddd"] = df["y_smooth_dd"] - df["y_smooth_dd"].shift(1)
    df["x_smooth_dddd"] = df["x_smooth_ddd"] - df["x_smooth_ddd"].shift(1)
    df["y_smooth_dddd"] = df["y_smooth_ddd"] - df["y_smooth_ddd"].shift(1)
    df.plot(x="x_smooth", y="y_smooth", style=".-")
    plt.axis("square")

In [10]:
if 0:
    df["xv_mm"] = df["x_smooth_d"] / 80 * 1000
    df["yv_mm"] = df["y_smooth_d"] / 80 * 1000
    df["xa_mm"] = df["x_smooth_dd"] / 80 * 1000 * 1000
    df["ya_mm"] = df["y_smooth_dd"] / 80 * 1000 * 1000
    df.plot(x="t", y=["xa_mm", "ya_mm"], style=".-")


In [11]:
if 0:
    from numpy.polynomial import polynomial as P
    i1 = 10000
    r = 2
    for n in range(10, 100, 5):
        x = df["t"].iloc[i1:i1 + n].values
        x = x - x[0]
        y = df["x_smooth"].iloc[i1:i1 + n].values
        c, stats = P.polyfit(x,y,2,full=True)
        y1 = P.polyval(x, c)
        print(("{:4d} | " + "{:11.2f} "*len(c) + "| {:10.8f}").format(n, *list(c), np.max(y1 - y) * 80))


In [12]:
from splipy import curve_factory
from scipy.interpolate import interp1d
    
t0 = df["t"].iloc[int(borders/2)]
t1 = df["t"].iloc[int(-borders/2)]
length = np.sum(df["d"].iloc[int(borders/2):int(-borders/2)])
rtol = 0.1/length

df["z"] = 0.0
xf = interp1d(df["t"], df[["x_smooth", "y_smooth", "e_smooth"]].T)
xxf = lambda x: xf(x).T

print(t0, t1, length)
c = curve_factory.fit(xxf, t0, t1, rtol=1e-4, atol=0.01)
print(len(c), c)
err = c.error(xxf)

print(err[1])
knots = c.knots()[0]
n_knots = []

# add first oversample
accel_t = all_segs.iloc[1]["t"]

idx = np.searchsorted(knots, accel_t)
print(accel_t)
kn_0 = knots[0]
kn_1 = knots[idx] + 0.010

rdt = kn_1 - kn_0
rn = int(ceil(rdt/0.005))
rsdt = rdt/rn

idx2 = np.searchsorted(knots, kn_1) + 3

for i in range(rn + 1):
    n_kn = kn_0 + rsdt * i
    ok = True
    for kn in knots[:idx2]:
        if abs(kn - n_kn) < 0.001:
            print("{} too close to {}".format(kn, n_kn))
            ok = False
            break
    if ok:    
        n_knots.append(n_kn)

# add last oversample
decel_t = all_segs.iloc[-1]["t"]

idx = np.searchsorted(knots, decel_t) - 1
print(decel_t)
print(knots[idx - 1])
print(knots[idx])
print(knots[idx + 1])
kn_0 = knots[idx] - 0.010
kn_1 = knots[-1]

rdt = kn_1 - kn_0
rn = int(ceil(rdt/0.005))
rsdt = rdt/rn

idx2 = np.searchsorted(knots, kn_0) - 3

for i in range(rn + 1):
    n_kn = kn_1 - rsdt * i
    ok = True
    for kn in knots[idx2:]:
        if abs(kn - n_kn) < 0.002:
            print("{} too close to {}".format(kn, n_kn))
            ok = False
            break
    if ok:    
        n_knots.append(n_kn)
                
print(n_knots)
knots.extend(n_knots)
knots.sort()
knots = [knots[0]] * 3 + knots + [knots[-1]] * 3

b = curve_factory.BSplineBasis(4, knots)
# do interpolation and return result
t = np.array(b.greville())
crv = curve_factory.interpolate(xxf(t), b, t)
print(len(crv), crv)
(err2, max_err) = crv.error(xxf)
print(max_err)
c = crv

-0.059999999999999956 29.366000000012907 280884.0841811873
1403 p=4, [-0.06  -0.06  -0.06  ... 29.366 29.366 29.366]
[[-7.89500000e+00  1.66070000e+01  0.00000000e+00]
 [-7.88888380e+00  1.66210332e+01  7.50395817e-04]
 [-7.92558099e+00  1.65368342e+01 -3.75197909e-03]
 ...
 [ 5.71561555e+00  3.34336538e+01 -1.63792666e+02]
 [ 5.65021544e+00  3.33682078e+01 -1.63788029e+02]
 [ 5.66100000e+00  3.33790000e+01 -1.63788780e+02]]
0.4588216089277575
0.009588824237876901
-0.059999999999999956 too close to -0.059999999999999956
0.013565000000032315 too close to 0.01373382352944029
29.261333076451983
29.222956944457287
29.243391666679518
29.274043750012865
29.366000000012907 too close to 29.366000000012907
29.33534791667956 too close to 29.336531481494376
29.274043750012865 too close to 29.272683024704225
29.243391666679518 too close to 29.243214506185694
[-0.05508441176470394, -0.05016882352940792, -0.045253235294111904, -0.04033764705881589, -0.03542205882351987, -0.03050647058822386, -0.0255

In [13]:
%matplotlib notebook

if 1:
    df1 = pd.DataFrame()

    df1["t"] = df["t"].iloc[70:-70]
    res = c(df1["t"].values)
    res2 = xxf(df1["t"].values)

    df1["crv_x"] = res[:, 0]
    df1["crv_y"] = res[:, 1]
    df1["crv_e"] = res[:, 2]
    df1["x"] = res2[:, 0]
    df1["y"] = res2[:, 1]
    df1["e"] = res2[:, 2]
    df1["dx"] = df1["crv_x"] - df1["x"]
    df1["dy"] = df1["crv_y"] - df1["y"]
    df1["d"] = np.linalg.norm(df1[["dx", "dy"]], axis=1)
    df1["de"] = (df1["crv_e"] - df1["e"])*837
    res3 = c(c.knots()[0])
    df2 = pd.DataFrame()
    df2["kn_x"] = res3[:, 0]
    df2["kn_y"] = res3[:, 1]
    df2["kn_e"] = res3[:, 2]
    df2["t"] = c.knots()[0]

    if 1:
        ax = df1.plot(x="crv_x", y="crv_y", style=".-")
        df1.plot(x="x", y="y", style=".-", ax=ax)
        df2.plot(x="kn_x", y="kn_y", style="*", ax=ax)
        plt.axis("square")
    else:   
        df1.plot(x="t", y="d")



<IPython.core.display.Javascript object>

In [14]:
if 1:
    df2["dt"] = df2["t"] - df2["t"].shift(1)
    df2[df2["dt"] < 10.010]

In [15]:
if 1:
    df1["x_d"] = df1["crv_x"] - df1["crv_x"].shift(1)
    df1["y_d"] = df1["crv_y"] - df1["crv_y"].shift(1)
    df1["e_d"] = df1["crv_e"] - df1["crv_e"].shift(1)
    df1["x_dd"] = df1["x_d"] - df1["x_d"].shift(1)
    df1["y_dd"] = df1["y_d"] - df1["y_d"].shift(1)
    df1["e_dd"] = df1["e_d"] - df1["e_d"].shift(1)
    df1["x_ddd"] = df1["x_dd"] - df1["x_dd"].shift(1)
    df1["y_ddd"] = df1["y_dd"] - df1["y_dd"].shift(1)
    df1["e_ddd"] = df1["e_dd"] - df1["e_dd"].shift(1)
    df1["x_dddd"] = df1["x_ddd"] - df1["x_ddd"].shift(1)
    df1["y_dddd"] = df1["y_ddd"] - df1["y_ddd"].shift(1)
    df1["e_dddd"] = df1["e_ddd"] - df1["e_ddd"].shift(1)

    df1["xv_mm"] = df1["x_d"] * 1000
    df1["yv_mm"] = df1["y_d"] * 1000
    df1["ev_mm"] = df1["e_d"] * 1000
    df1["xa_mm"] = df1["x_dd"] * 1000 * 1000
    df1["ya_mm"] = df1["y_dd"] * 1000 * 1000
    df1["ea_mm"] = df1["e_dd"] * 1000 * 1000

    ax = df1.plot(x="t", y=["xv_mm", "yv_mm"])
    df2["z"] = 0
    df2.plot(x="t", y="z", style="*", ax=ax)


<IPython.core.display.Javascript object>

In [16]:
from valurap2.profile import ProfileSegment
from valurap2.emulate import emulate
from valurap2.path_planning import PathPlanner
from math import ceil

apg_x = 0
apg_y = 2
apg_e = 4
apg_x_s = pp.apg_states[apg_x]
apg_y_s = pp.apg_states[apg_y]
apg_e_s = pp.apg_states[apg_e]
t_k = apg_x_s.t_k

knots = c.knots()[0]

knots.append(0.0)
knots.sort()

if 0:
    n_knots = []
    prev_knot = None

    for k in knots:
        if prev_knot is None:
            prev_knot = k
            n_knots.append(k)
            continue
        dt = k - prev_knot
        if dt < 10.050:
            prev_knot = k
            n_knots.append(k)
            continue

        splits = int(dt / 0.020)
        #print("splitting", prev_knot, k, dt, splits, sdt)
        sdt = dt / splits
        for i in range(1, splits):
            n_knots.append(prev_knot + sdt * i)
            #print("   ", prev_knot + sdt * i)
        n_knots.append(k)
        #print("   ", k)
        prev_knot = k


    knots = n_knots

x_vals = c(knots)
v_vals = c.derivative(knots, d=1)
a_vals = c.derivative(knots, d=2)
j_vals = c.derivative(knots, d=3)

tg_x, tg_y, tg_e = x_vals[0]
segs = [
    ProfileSegment(apg=apg_x, x=int(tg_x / apg_x_s.x_k), v=0, a=0),
    ProfileSegment(apg=apg_y, x=int(tg_y / apg_y_s.x_k), v=0, a=0),
    ProfileSegment(apg=apg_e, x=int(tg_e / apg_e_s.x_k), v=0, a=0)
]

segments = []
sub_profile = [[5, segs]]
emulate(
    sub_profile,
    apg_states=pp.apg_states,
    accel_step=pp.accel_step,
    step_bit=pp.step_bit,
    no_tracking=True,
)

segments.extend(sub_profile)

last_e_epoch = 0.0
last_x = apg_x_s.x_float()
last_vx = apg_x_s.v_float()
last_y = apg_y_s.x_float()
last_vy = apg_y_s.v_float()
last_e = apg_e_s.x_float() + last_e_epoch
last_ve = apg_e_s.v_float()
last_ax = apg_x_s.a_float()
last_ay = apg_y_s.a_float()
last_ae = apg_e_s.a_float()

for i, t in enumerate(knots):
    int_t = int(round(t / t_k))
    
    if i == 0:
        print(t)
        last_int_t = int_t
        continue
    
    int_dt = int_t - last_int_t
    dt = int_dt * t_k
    
    tg_x, tg_y, tg_e = x_vals[i]
    tg_vx, tg_vy, tg_ve = v_vals[i]
    exp_ax, exp_ay, exp_ae = a_vals[i - 1]
    exp_jx, exp_jy, exp_je = j_vals[i - 1]

    def calc_aj(dt, x, last_x, v, last_v, a_k, j_k):
        dx = x - last_x

        ax = (6 * dx - 2 * dt * v - 4 * dt * last_v) / dt / dt
        int_ax = round(ax / a_k)

        ax = int_ax * a_k
        jx = (6 * dx - 6 * dt * last_v - 3 * ax * dt * dt) / dt / dt / dt

        int_jx = round(jx / j_k)
        jx = int_jx * j_k

        return ax, int_ax, jx, int_jx

    ax, int_ax, jx, int_jx = calc_aj(dt, tg_x, last_x, tg_vx, last_vx, apg_x_s.a_k, apg_x_s.j_k)
    ay, int_ay, jy, int_jy = calc_aj(dt, tg_y, last_y, tg_vy, last_vy, apg_y_s.a_k, apg_y_s.j_k)
    ae, int_ae, je, int_je = calc_aj(dt, tg_e, last_e, tg_ve, last_ve, apg_e_s.a_k, apg_e_s.j_k)

    profile = [
        [
            int_dt,
            [
                ProfileSegment(apg=apg_x, a=int(int_ax), j=int(int_jx)),
                ProfileSegment(apg=apg_y, a=int(int_ay), j=int(int_jy)),
                ProfileSegment(apg=apg_e, a=int(int_ae), j=int(int_je)),
            ],
        ]
    ]
    emulate(profile, apg_states=pp.apg_states, accel_step=pp.accel_step, no_tracking=True)
    segments.extend(profile)

    last_x = apg_x_s.x_float()
    last_vx = apg_x_s.v_float()
    last_y = apg_y_s.x_float()
    last_vy = apg_y_s.v_float()
    last_e_new = apg_e_s.x_float() + last_e_epoch
    last_e_delta = (last_e_new - last_e) * 837
    if abs(last_e_delta) > 30000:
        if last_e_delta > 0:
            last_e_epoch -= 65536.0 / 837
        else:
            last_e_epoch += 65536.0 / 837
        print("new_e_epoch",last_e_epoch)
    last_e = apg_e_s.x_float() + last_e_epoch
    last_ve = apg_e_s.v_float()
    last_int_t += int_dt

    print("{:7.3f} {:4d} x: {:5.2f}/{:5.2f} y: {:5.2f}/{:5.2f} e: {:5.2f}/{:5.2f} a: {:9d} {:9d} {:9d} j: {:6d} {:6d} {:6d}".format(
        t, int_dt,
        (last_x - tg_x) * 80, last_vx - tg_vx,
        (last_y - tg_y) * 80, last_vy - tg_vy,
        (last_e - tg_e) * 837, last_ve - tg_ve,
        int(int_ax),
        int(int_ay),
        int(int_ae),
        int(int_jx),
        int(int_jy),
        int(int_je),
    ))

    last_ax = apg_x_s.a_float()
    last_ay = apg_y_s.a_float()
    last_ae = apg_e_s.a_float()

segs = [
    ProfileSegment(apg=apg_x, v=0, a=0),
    ProfileSegment(apg=apg_y, v=0, a=0),
    ProfileSegment(apg=apg_e, v=0, a=0)
]

profile = [[5, segs]]  
emulate(profile, apg_states=pp.apg_states, accel_step=pp.accel_step, no_tracking=True)
segments.extend(profile)

last_vx = apg_x_s.v_float()
last_vy = apg_y_s.v_float()
last_ve = apg_e_s.v_float()
last_ax = apg_x_s.a_float()
last_ay = apg_y_s.a_float()
last_ae = apg_e_s.a_float()

print(last_vx, last_vy, last_ve, last_ax, last_ay, last_ae)
print(len(segments))

Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
-0.059999999999999956
 -0.055  246 x: -0.00/ 0.00 y:  0.00/ 0.00 e:  0.00/-0.00 a:         0         0         0 j:      0      0      0
 -0.050  246 x: -0.00/-0.00 y: -0.00/-0.00 e:  0.00/ 0.00 a:        -1        -1         0 j:      0      0      0
 -0.045  245 x:  0.00/ 0.00 y:  0.00/ 0.00 e:  0.00/-0.00 a:         9        11         0 j:      0      0      0
 -0.040  246 x: -0.00/-0.00 y:  0.00/ 0.00 e:  0.00/ 0.00 a:       -57       -78         1 j:      0      1      0
 -0.035  246 x:  0.00/ 0.00 y: -0.00/-0.00 e: -0.00/-0.00 a:       327      -257       -10 j:     -2      1      0
 -0.031  246 x:  0.00/ 0.00 y:  0.00/ 0.00 e: -0.00/-0.00 a:      -324       224        68 j:      3      0     -1
 -0.026  245 x:  0.00/ 0.00 y:  0.00/-0.00 e: -0.00/-0.00 a:        18        86       327 j:     -3     -7     -2
 -0.023  119 x: -0.00/-0.00 y: -0.00/-0.00 e:  0.00/ 0.00 a:     -1623     -1847

  7.100 1226 x: -0.02/-0.01 y:  0.04/ 0.02 e: -0.02/-0.00 a:   -520121   3894291  -1335150 j:   5599 -14153   6734
  7.125 1226 x:  0.01/ 0.01 y: -0.01/-0.02 e:  0.00/ 0.00 a:   6368523 -13496907   6943050 j:  -9826  16005  -8273
  7.149 1227 x: -0.03/-0.02 y: -0.04/-0.01 e:  0.02/ 0.00 a:  -5639488   5993409  -3124787 j:  13934   3541   -559
  7.174 1226 x:  0.01/ 0.02 y:  0.03/ 0.03 e: -0.01/-0.00 a:  11550118  10319261  -3784874 j: -13929 -19161   7622
  7.186  613 x: -0.01/-0.01 y:  0.01/-0.02 e: -0.00/ 0.00 a:  -5563954 -13307965   5612628 j:  20437  22532  -8120
  7.198  613 x: -0.00/ 0.01 y:  0.02/ 0.01 e: -0.01/-0.00 a:   6983932    380915    690188 j: -13881 -16948   6891
  7.247 2452 x: -0.02/-0.01 y: -0.08/-0.03 e:  0.03/ 0.00 a:  -1476047 -10002658   4917838 j:   1742   9989  -4881
  7.272 1226 x: -0.03/-0.01 y: -0.02/ 0.01 e:  0.01/-0.00 a:   2819990  14628729  -7109831 j:   5137 -10491   4522
  7.284  613 x: -0.01/ 0.01 y:  0.03/ 0.02 e: -0.01/-0.00 a:   9320482   1972649

 16.860  682 x:  0.08/ 0.05 y:  0.04/ 0.03 e:  0.04/ 0.00 a:   1805014   2175741    616214 j: -65523 -34089 -34894
 16.874  681 x:  0.10/-0.00 y:  0.10/ 0.03 e:  0.07/ 0.00 a: -43412101 -21275041 -23419103 j:    373 -41001 -10840
 16.887  681 x: -0.03/-0.08 y: -0.03/-0.09 e: -0.01/-0.01 a: -44022213 -49956775 -31361511 j: 109919 120911  72749
 16.901  681 x: -0.09/-0.01 y: -0.09/-0.01 e: -0.06/-0.00 a:  30621973  32160846  17998552 j:  16017  13427  16969
 16.914  681 x: -0.06/ 0.03 y:  0.00/ 0.06 e: -0.03/ 0.00 a:  42041150  41814921  29882856 j: -34371 -83749 -30620
 16.928  681 x: -0.01/ 0.02 y: -0.06/-0.06 e:  0.02/ 0.00 a:  19139700 -15073420   9328814 j: -31550  79589 -33967
 16.942  681 x: -0.11/-0.07 y: -0.12/-0.01 e:  0.08/ 0.00 a:  -2156822  39432055 -13869712 j:  91454  16472 -34135
 16.962 1022 x:  0.06/ 0.11 y:  0.05/ 0.09 e: -0.03/-0.01 a:  60352006  51103631 -37346962 j: -100442 -83470  58485
 16.983 1022 x:  0.06/-0.04 y:  0.12/-0.00 e: -0.08/ 0.00 a: -42455949 -3434181

 25.586 1022 x: -0.09/ 0.03 y: -0.14/-0.01 e: -0.09/ 0.00 a:  41071435  33182528  26550479 j: -22629   8394  -2380
 25.606 1022 x: -0.07/-0.00 y: -0.04/ 0.05 e:  0.03/ 0.00 a:  18260949  42185585  24408981 j:   3024 -41262 -41918
 25.626 1021 x: -0.12/-0.02 y: -0.11/-0.05 e:  0.09/ 0.00 a:  21708926    321475 -18498754 j:  16219  40687  -8554
 25.647 1022 x: -0.14/-0.00 y: -0.14/ 0.00 e:  0.10/ 0.00 a:  38590927  42116540 -27459054 j:    283  -3925   -962
 25.688 2044 x:  0.14/ 0.09 y:  0.13/ 0.08 e: -0.09/-0.01 a:  38998590  38226287 -28526542 j: -37872 -37022  27153
 25.708 1021 x:  0.14/-0.00 y:  0.15/ 0.01 e: -0.10/-0.00 a: -38594241 -37616691  27079994 j:    807  -5440    414
 25.729 1022 x:  0.10/-0.01 y:  0.04/-0.05 e: -0.08/ 0.00 a: -38042049 -43462993  27682901 j:  12661  41754  -6791
 25.749 1022 x:  0.06/-0.01 y:  0.10/ 0.04 e:  0.04/ 0.00 a: -25490700  -1072474  21050922 j:  12648 -35871 -41514
 25.769 1021 x:  0.12/ 0.03 y:  0.13/-0.00 e:  0.09/ 0.00 a: -12891718 -38148452

In [17]:
df_emu = emulate(segments[:300], apg_states=pp.apg_states, accel_step=pp.accel_step, no_tracking=False)


In [18]:
df_emu["x_mm"] = df_emu["apg0_x"]
df_emu["y_mm"] = df_emu["apg2_x"]
df_emu["e_mm"] = df_emu["apg4_x"]
df_emu["x_steps"] = df_emu["apg0_x"] * 80
df_emu["y_steps"] = df_emu["apg2_x"] * 80
df_emu["e_steps"] = df_emu["apg4_x"] * 837
df_emu["vx_mm"] = df_emu["apg0_v"]
df_emu["vy_mm"] = df_emu["apg2_v"]
df_emu["ve_mm"] = df_emu["apg4_v"]
last_t = df_emu["t"].iloc[-1]
orig_n = int(last_t * 1000)
print(orig_n)


knots = c.knots()[0]
res3 = c(knots)
df2 = pd.DataFrame()
df2["kn_x"] = res3[:, 0]
df2["kn_y"] = res3[:, 1]
df2["kn_e"] = res3[:, 2]
df2["t"] = c.knots()[0]
knots_n = np.searchsorted(knots, last_t)

#res4 = c(df_emu["t"] + knots[0])
#df_emu["c_x"] = res4[:, 0]
#df_emu["c_y"] = res4[:, 1]
#df_emu["c_e"] = res4[:, 2]

ax = df_emu.plot(x="x_mm", y="y_mm", style=".-", zorder=1)
#df_emu.plot(x="c_x", y="c_y", style=".-", zorder=2, ax=ax)
df[:orig_n].plot(x="x_smooth", y="y_smooth", style=".-", zorder=2, ax=ax)
df2[:knots_n].plot(x="kn_x", y="kn_y", style="*", ax=ax, zorder=3)
plt.axis("square")

7332


<IPython.core.display.Javascript object>

(-15.747780487680718,
 49.44713309211586,
 -16.545892582561688,
 48.649020997234885)

In [19]:
df_emu.plot(x="t", y=["vx_mm", "vy_mm"], style=".-", zorder=2)


<IPython.core.display.Javascript object>

In [20]:
from valurap2 import buf_commands, rest_client

cb = buf_commands.CommandBuffer(debug=False)
cb.add_segments_head(pp=pp)
cb.add_segments(segments)
cb.add_segments_tail()

In [21]:
import time

if 1:
    cl = rest_client.Client(emu=False)
    cl.abort()
    time.sleep(1)
    cl.home()
    cl.wait_idle()
    cl.moveto(Y=50, X1=0)
    cl.wait_idle()

#c.exec_binary(cb)

ConnectionError: HTTPConnectionPool(host='orange2', port=5000): Max retries exceeded with url: /api?cmd=abort (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x118d6a3d0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
if 1:
    cl.moveto(Z=9.3+10)
    cl.wait_idle()
    cl.moveto(X1=100)
    cl.wait_idle()
    cl.moveto(E1=-10)
    cl.wait_idle()
    cl.moveto(X1=0)
    cl.wait_idle()
    cl.moveto(Z=9.3+0.3)
    cl.wait_idle()
    cl.moveto(X1=0)
    cl.wait_idle()

    for i in range(1):
        cl.enable(mode="print", axes="e1,e2")
        cl.exec_binary(cb.buffer)
        cl.wait_idle()
    cl.moveto(Z=9.3+10)
    cl.wait_idle()
